<a href="https://colab.research.google.com/github/rodrigoreisbr-tech/data_science/blob/master/(12ABD)%20PySpark_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.1.1 with hadoop 3.2, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.

Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz


!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time 

to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

Run a local spark session to test your installation:

In [ ]:
import findspark

findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Congrats! Your Colab is ready to run Pyspark.

In [ ]:
spark

In [ ]:
from google.colab import files
files.upload()

# Análise do arquivo (results.csv)

Realizar análise do arquivo results.csv

O dataset results.csv é um conjunto de dados que inclui resultados de jogos internacionais de futebol. Os jogos são estritamente internacionais masculinos e os dados não incluem Jogos Olímpicos ou jogos em que pelo menos uma das equipes foi a seleção B do país, sub-23 ou uma seleção da liga. O dataset foi retirado do Kaggle.

Metadados:

date - data da partida
home_team - time mandante
away_team - time visitante
home_score - placar do time mandante
away_score - placar do time visitante
tournament - torneio
city - cidade onde a partida foi realizada
country - país onde a partida foi realizada
neutral - desconsiderar

Importar o arquivo no Spark e responder as perguntas abaixo utilizando python ou SQL.

A atividade por ser realizada instalando o Spark na máquina local, utilizando docker ou o ambiente Databricks Community.
Para cada questão, informar a resposta e o código utilizado para chegar
ao resultado.

1 - Quantos registro existem na base?

2 - Quantas equipes únicas mandantes existem na base?

3 - Quantas vezes as equipes mandantes saíram vitoriosas?

4 - Quantas vezes as equipes visitantes saíram vitoriosas?

5 - Quantas partidas resultaram em empate?

6 - Quantas partidas foram realizadas em cada país?

7 - Qual país teve mais partidas?

8 - Qual a partida com maior número de gols?

9 - Qual a maior goleada?

10 - Quantos jogos ocorreram no Brasil?

  # Tranformando o arquivo em uma dataset

O dataset results.csv é um conjunto de dados que inclui resultados de jogos internacionais de futebol. Os jogos são estritamente internacionais masculinos e os dados não incluem Jogos Olímpicos ou jogos em que pelo menos uma das equipes foi a seleção B do país, sub-23 ou uma seleção da liga. O dataset foi retirado do Kaggle.


In [ ]:
#importando os dados do results.csv
dataset = spark.read.csv('results.csv',inferSchema=True, header =True)
type(dataset)

pyspark.sql.dataframe.DataFrame

# Verificando os metadados

Segue abaixo os metadados do dataset.

In [ ]:
dataset.printSchema()

root
 |-- date: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)



Dicionario de dados

*   date - data da partida
*   home_team - time mandante
*   away_team - time visitante
*   home_score - placar do time mandante
*   away_score - placar do time visitante
*   tournament - torneio
*   city - cidade onde a partida foi realizada
*   country - país onde a partida foi realizada
*   neutral - desconsiderar

In [ ]:
dataset.show(10)

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  false|
|1876-03-25| Scotland|    Wales|         4|         0|  Friendly|Glasgow|Scotland|  false|
|1877-03-03|  England| Scotland|         1|         3|  Friendly| London| England|  false|
|1877-03-05|    Wales| Scotland|         0|         2|  Friendly|Wrexham|   Wales|  false|

##1 - Quantos registro existem na base?

No total temos 40.839 registros no dataset.



In [ ]:
dataset.count()

40839

##2 - Quantas equipes únicas mandantes existem na base?

Existem 309 equipes diferentes nas olimpiadas.


In [ ]:
dataset.select('home_team').distinct().orderBy('home_team').count()


309

##3 - Quantas vezes as equipes mandantes saíram vitoriosas?

In [ ]:
dataset.filter(dataset.home_score > dataset.away_score).count()

19864

##4 - Quantas vezes as equipes visitantes saíram vitoriosas?

In [ ]:
dataset.filter(dataset.home_score < dataset.away_score).count()

11544

## 5 - Quantas partidas resultaram em empate?


In [ ]:
dataset.filter(dataset.home_score == dataset.away_score).count()

9431

##6 - Quantas partidas foram realizadas em cada país?

In [ ]:
 dataset \
    .groupBy('country') \
    .agg({'country' : 'count'}) \
    .orderBy('count(country)',ascending = False) \
    .show()

+--------------------+--------------+
|             country|count(country)|
+--------------------+--------------+
|       United States|          1144|
|              France|           801|
|             England|           687|
|            Malaysia|           644|
|              Sweden|           637|
|             Germany|           581|
|              Brazil|           529|
|               Spain|           517|
|            Thailand|           483|
|               Italy|           480|
|         Switzerland|           477|
|             Austria|           475|
|United Arab Emirates|           472|
|        South Africa|           470|
|               Qatar|           467|
|         South Korea|           453|
|           Argentina|           449|
|             Hungary|           431|
|               Chile|           405|
|             Belgium|           396|
+--------------------+--------------+
only showing top 20 rows



##7 - Qual país teve mais partidas?

In [ ]:
dataset \
    .groupBy('country') \
    .agg({'country' : 'count'}) \
    .orderBy('count(country)',ascending = False) \
    .show(1)

+-------------+--------------+
|      country|count(country)|
+-------------+--------------+
|United States|          1144|
+-------------+--------------+
only showing top 1 row



##8 - Qual a partida com maior número de gols?

In [ ]:

#df = dataset.withColumn('total', sum(dataset[total] for total in ('home_score','away_score')))
df.agg({'total' : 'max'}).show()

df.registerTempTable('table')

#df.filter(df.total.max).show()



+----------+
|max(total)|
+----------+
|        31|
+----------+



##9 - Qual a maior goleada?


In [ ]:
dataset.agg({'home_score' : 'max'}).show()

+---------------+
|max(home_score)|
+---------------+
|             31|
+---------------+



##10 - Quantos jogos ocorreram no Brasil?

In [ ]:
dataset.filter(dataset.country.like('Brazil')).count()

529